### Some analysis

In [1]:
from officelib.xllib import *
from officelib.const import xlconst as xlc
import os

In [2]:
keys = {
    'temperature.pv',
    'agitation.pv',
    'ph.outputDown',
    'ph.manDown',
    'do.manUp',
    'do.outputDown',
    'do.pv',
    'do.manDown',
    'do.outputUp',
    'maingas.man',
    'maingas.mode',
    'maingas.pv',
    'MFCs.air',
    'MFCs.n2',
    'MFCs.o2',
    'MFCs.co2',
    'Elapsed',
    'Time'
}
def hide_columns(ws):
    for col in ws.UsedRange.Columns:
        c = col.Cells(1,1)
        if c.Value not in keys:
            col.Hidden=True

In [13]:
def data_range(ws, name):
    c1 = ws.Cells.Find(name)
    if c1 is None: raise ValueError(name)
    c1 = c1.Offset(2,1)
    c2 = c1.End(xlc.xlDown)
    return c1, c2

class SeriesProxy():
    def __init__(self, ws, var, xvar="Elapsed", yax=1, name=None):
        self.ws = ws
        self.var = var
        self.yax = yax
        self.s = None
        self.xvar = xvar
        self.name = name if name else var
        
    def create(self, chart):
        e1, e2 = data_range(self.ws, self.xvar)
        c1, c2 = data_range(self.ws, self.var)
        cr = self.ws.Cells.Range
        self.xr = cr(e1, e2)
        self.yr = cr(c1, c2)
        self.s = CreateDataSeries(chart, self.xr, self.yr, self.name)
        if self.yax != 1:
            self.s.AxisGroup = self.yax
        
    def set_name(self, n):
        self.name = n
        self.s.Name = n 
        
class ChartProxy():
    def __init__(self, ws, chart=None, *args):
        self.ws = ws
        self.chart = chart or CreateChart(ws, *args)
        self.chart.ChartStyle = 240  # Excel 2017 style
        self.series = []
        
    def add_series(self, var, xvar="Elapsed", yax=1, name=None):
        s = SeriesProxy(self.ws, var, xvar, yax, name)
        self.make_series([s])
        return s
        
    def make_series(self, series):
        for s in series:
            s.create(self.chart)
            self.series.append(s)
            
    def _axlabel(self, ax, t):
        ax.HasTitle=True
        ax.AxisTitle.Text=t
    
    def xlabel(self, t):
        xax = self.chart.Axes(xlc.xlCategory, xlc.xlPrimary)
        self._axlabel(xax, t)
        
    def ylabel(self, t):
        yax = self.chart.Axes(xlc.xlValue, xlc.xlPrimary)
        self._axlabel(yax, t)
        
    def ylabel2(self, t):
        yax = self.chart.Axes(xlc.xlValue, xlc.xlSecondary)
        self._axlabel(yax, t)
        
    def title(self, t):
        self.chart.HasTitle = True
        self.chart.ChartTitle.Text = t
        
    def copyto(self, r):
        # chart parent -> ChartObject
        self.chart.Parent.Copy()
        r.Paste()
         
def _listify(args):
    l = []
    if isinstance(args, str):
        return [args]
    for a in args:
        if isinstance(a, (list,tuple)):
            l.extend(_listify(a))
        else:
            l.append(a)
    return l
    
def make_chart(names, title=None, xlabel=None, ylabel=None, xname="Elapsed", *, uws=None):
    uws = uws or ws
    c = ChartProxy(uws)
        
    names = _listify(names)
    c.make_series(SeriesProxy(uws,n, xname) for n in names)
    
    # optional setup
    if title: c.title(title)
    if xlabel: c.xlabel(xlabel)
    if ylabel: c.ylabel(ylabel)
    
    return c

### Example code for now to use functions

In [14]:
# # MFCs
# mfcc = [
#     "MFCs.Air",
#     "MFCs.O2",
#     "MFCs.N2",
#     "MFCs.CO2"
# ]
# make_chart(mfcc, "MFC Flow", "Time(s)", "Flow Rate (LPM)")
# make_chart("do.pv", "DO PV", "Time(s)", "DO PV (%)")

#### Some analysis

In [15]:
xl = Excel()
for wb in xl.Workbooks: wb.Close(False)
    
def add_af1(name, new, form):
    e1 = ws.Cells.Find(name)
    e1.Offset(1,2).EntireColumn.Insert()
    e1.Offset(1,2).Value = new
    
    c1 = e1.Offset(2,2)
    c2 = e1.End(xlc.xlDown).Offset(1,2)
    c1.Value = form % e1.Offset(2,1).GetAddress(0,0)
    afr = ws.Cells.Range(c1, c2)
    c1.AutoFill(afr)
    return c1, c2
    

def nch(ws, n):
    c = CreateChart(ws)
    c.Location(xlc.xlLocationAsNewSheet, n)
    return ws.Parent.Charts(n)
    
def gfp(f): return os.path.abspath(f)
    
def analyze2():
    # globals make debugging easy
    global wb, ws
    os.chdir('C:\\Users\\natha\\Documents\\test\\do_3L_verif_hs_new 170727\\')
    os.makedirs("analyzed", exist_ok=True)
    xl.DisplayAlerts=False
    for f in filter(os.path.isfile, os.listdir()):
        print("Analyzing", f)
        wb = xl.Workbooks.Open(gfp(f))
        ws = wb.Worksheets(1)
        e1, e2 = add_af1("Elapsed", "Elapsed.hr", "=%s/3600")
        ws.Cells.Range(e1, e2).NumberFormat = "0.000"
        
        typ = " ".join(f.split()[-3:])[:-4]
            
        c = make_chart('do.pv', typ, "Time(hr)", 
                       "DO PV (%)", "Elapsed.hr", uws=ws)
        c.add_series("do.outputUp", "Elapsed.hr", 2, "O2 Request")
        c.add_series("do.outputDown", "Elapsed.hr", 2, "N2 Request") 
        
        c2 = make_chart([
                "MFCs.air",
                "MFCs.O2",
                "MFCs.N2",
            ], "MFC Gas Flow", "Time(hr)", "Flow Rate(LPM)", "Elapsed.hr", uws=ws)
        
        if typ == "100 to 150":
            y1, y2 = 140, 160
        elif typ == "100 to 50":
            y1, y2 = 40, 60
        else:
            y1, y2 = None, None
        
        FormatAxesScale(c.chart, None, None, y1, y2, Y2AxisMin=0, Y2AxisMax=100)
        wb.SaveAs(gfp("analyzed\\"+f.replace(".csv", ".xlsx")), FileFormat=xlc.xlOpenXMLWorkbook)
        if "4." not in f:
            wb.Close(True)

In [16]:
analyze2()

Analyzing 3L_do_pid 4.-20.55.40 repeat 100 to 50.csv
